### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score


# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [2]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [3]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
tfidfvect = TfidfVectorizer(stop_words='english')

In [4]:
# en el atributo `data` accedemos al texto
print(newsgroups_train.data[0])

I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.


In [5]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [6]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'Cantidad de documentos: {X_train.shape[0]}')
print(f'Tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101322)
Cantidad de documentos: 11314
Tamaño del vocabulario (dimensionalidad de los vectores): 101322


In [7]:
# una vez fiteado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25717

In [8]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [9]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [10]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [11]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [12]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [13]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.5377831 , 0.49482167, ..., 0.        , 0.        ,
       0.        ])

In [14]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([4811, 6635, 3596, ..., 9160, 1781,  827])

In [15]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [16]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.politics.misc'

In [17]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


### Modelo de clasificación Naïve Bayes

In [18]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [19]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [20]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.6467991505900852

### Consigna del desafío 1

**Cada experimento realizado debe estar acompañado de una explicación o interpretación de lo observado.**

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Construir un modelo de clasificación por prototipos (tipo zero-shot). Clasificar los documentos de un conjunto de test comparando cada uno con todos los de entrenamiento y asignar la clase al label del documento del conjunto de entrenamiento con mayor similaridad.

**3**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**NO cambiar el hiperparámetro ngram_range de los vectorizadores**.

**4**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares.

**Elegir las palabras MANUALMENTE para evitar la aparición de términos poco interpretables**.


# Desafio

## 1. Vectorizar documentos

In [51]:
# Elección de los 5 documentos de manera aleatoria

rng = np.random.default_rng() # Intente usar seed=42 pero es demasiado previsible y tal vez otros compañeros usen el mismo seed. Mejor cristalizar en un array los documentos a usar.
random_documents = rng.choice(len(newsgroups_train.data), size=5, replace=False)
print(random_documents)




[4643 8521 7937 3219 4669]


In [52]:
static_document_list = [4643, 8521, 7937, 3219, 4669]

In [56]:
# Funciones de ayuda
def get_similar_documents(idx, n = 5):
  print(f'\n\n--- DOCUMENTO ORIGINAL ---')
  print(f'DOC: {idx}')
  print(f'CLASS: {newsgroups_train.target_names[y_train[idx]]}\n\n')

  print(newsgroups_train.data[idx])

  cossim = cosine_similarity(X_train[idx], X_train)[0]

  # Obtener los índices de los más similares (de mayor a menor)
  similar_indices = np.argsort(cossim)[::-1]

  # Obtener los Top (N+1) índices (el primero es el doc original mismo)
  top_indices = similar_indices[1:(n + 1)]
  top_scores = cossim[top_indices]

  for i, idx in enumerate(top_indices):
    print(f'\n\n--- DOCUMENTO SIMILAR({i}) ---')
    print(f'DOC: {idx}')
    print(f'CLASS: {newsgroups_train.target_names[y_train[idx]]}\n\n')

    print(newsgroups_train.data[idx])

  return top_indices, top_scores



In [57]:
# Utilice esta funcion para intentar entender que palabras son mas relevantes
def palabras_mas_importanes(idx, N):
  feature_names = tfidfvect.get_feature_names_out()

  doc_vector = X_train[idx]
  doc_scores = doc_vector.toarray().flatten()
  top_indices = np.argsort(doc_scores)[::-1]
  N = 10
  print(f"Top {N} palabras más relevantes para este documento:\n")

  for i in top_indices[:N]:
      # Si el score es 0, ya no hay más palabras relevantes
      if doc_scores[i] == 0:
          break

      print(f"Palabra: {feature_names[i]:<20} | Score TF-IDF: {doc_scores[i]:.4f}")


### Primer documento

In [58]:
doc_0_similar, doc_0_scores = get_similar_documents(static_document_list[0])



--- DOCUMENTO ORIGINAL ---
DOC: 4643
CLASS: comp.windows.x


:    Where can I get xman source?  I would like to get the binaries for
: xman for an HP 9000/700, but I would settle for source.  
: 
: --
Try xport.lcs.mit.edu, in direcotry /contrib.
--


--- DOCUMENTO SIMILAR(0) ---
DOC: 3588
CLASS: comp.windows.x


Where can I get xman source?  I would like to get the binaries for
xman for an HP 9000/700, but I would settle for source.  

--


--- DOCUMENTO SIMILAR(1) ---
DOC: 3849
CLASS: comp.windows.x


   Where can I get xman source?  I would rather get 
xman for an HP 9000/700, but source will do.




--- DOCUMENTO SIMILAR(2) ---
DOC: 619
CLASS: comp.windows.x



  The OpenLook window manager source is available on the MIT contrib tapes
  or from  export.lcs.mit.edu  .I would suggest building this too, rather than
  using the version from OpenWindows. It is  olwm  v3.

				Regards,


--- DOCUMENTO SIMILAR(3) ---
DOC: 2437
CLASS: comp.windows.x


Subject says it all...
Anyone know w

Estos documentos son similares porque comparten un vocabulario técnico muy específico y raro.

TF-IDF asigna un puntaje altísimo a términos como `xman`, `source`, `binaries`, y el servidor `export.lcs.mit.edu`, precisamente porque estas palabras no aparecen en otros newsgroups (como rec.sport.baseball).

### Segundo documento

In [60]:
doc_1_similar, doc_1_scores = get_similar_documents(static_document_list[1])



--- DOCUMENTO ORIGINAL ---
DOC: 8521
CLASS: talk.religion.misc




OK: So you were a naive teen.


	Wrong: 1915 and if you do your homework, 1909.
But he was born LAST century (1883).



--- DOCUMENTO SIMILAR(0) ---
DOC: 6201
CLASS: talk.religion.misc





You might be sure, but you would also be wrong.



--- DOCUMENTO SIMILAR(1) ---
DOC: 749
CLASS: talk.politics.mideast


April 24th is approaching, and Armenians around the world
are getting ready to remember the massacres of their family members
by the Turkish government between 1915 and 1920.  
    At least 1.5 Million Armenians perished during that period,
and it is important to note that those who deny that this event
ever took place, either supported the policy of 1915 to exterminate
the Armenians, or, as we have painfully witnessed in Azerbaijan,
would like to see it happen again...
    Thank you for taking the time to read this post.


--- DOCUMENTO SIMILAR(2) ---
DOC: 1999
CLASS: soc.religion.christian



If babies are not s

Dado que el documento original es tan corto, me pregunto si TF-IDF está logrando capturar un tópico real.

Parece más probable que la similitud se deba a coincidencias en palabras clave aisladas que tienen un IDF alto (como `wrong`, `naive` o la fecha `1915`), en lugar de una similitud temática real. Con tan poco texto, el modelo parece estar amplificando el peso de estas pocas palabras sueltas.

### Documento 3

In [61]:
doc_2_similar, doc_2_scores = get_similar_documents(static_document_list[2])



--- DOCUMENTO ORIGINAL ---
DOC: 7937
CLASS: comp.graphics



This sounds wonderful, but it seems no one either wants to spend time doing
this, or they don't have the power to do so.  For example, I would like
to see a comp.graphics architecture like this:

comp.graphics.algorithms.2d
comp.graphics.algorithms.3d
comp.graphics.algorithms.misc
comp.graphics.hardware
comp.graphics.misc
comp.graphics.software/apps

However, that is almost overkill.  Something more like this would probably
make EVERYONE a lot happier:

comp.graphics.programmer
comp.graphics.hardware
comp.graphics.apps
comp.graphics.misc

It would be nice to see specialized groups devote to 2d, 3d, morphing,
raytracing, image processing, interactive graphics, toolkits, languages,
object systems, etc. but these could be posted to a relevant group or
have a mailing list organized.

That way when someone reads news they don't have to see these subject
headings, which are rather disparate:

System specific stuff ( should be und

Este es un caso "meta": los documentos son similares no porque hablen de algoritmos gráficos, sino porque discuten sobre la organización del propio newsgroup comp.graphics.

Términos como comp.graphics, comp.graphics.programmer, CFV (Call for Votes) y newsgroup tienen un score TF-IDF altísimo.

El documento original (7937) y el similar (8085) son idénticos en su intención (reorganizar el foro), mientras que los otros son fragmentos de discusiones sobre qué pertenece o no a un newsgroup. TF-IDF acertó al agruparlos por este tema de "discusión sobre Usenet".

### Documento 4

In [62]:
doc_3_similar, doc_3_scores = get_similar_documents(static_document_list[3])



--- DOCUMENTO ORIGINAL ---
DOC: 3219
CLASS: talk.politics.misc


If the heading is true, Mr. Frank should be ashamed of himself.

Nothing makes me gag more than people who don't respect the
rights of others to voice their opinions.  My idol Lenny Bruce
once commented about "that asshole Time Magazine" when they
advocated censorship of his material.  Time actually sided
with the cops' and their arresting of Bruce at his shows,
whereby he routinely would say "cocksucker", then the cops
would rush the stage to arrest him.  My, how the times haven't
changed...

I can't help but think of how Lenny would be received in today's
politically correct arena.  Heck, I even support the right of
neo nazis to speak their opinions and march down the streets.

And before Mr. Frank or anyone else makes any wisecracks about
anti-Semitism...I'm Jewish, a longtime member of AIPAC and the JNF,
and have contributed over $1000 apiece to these fine groups.  I'm
a regular contributor to every pro-Israel group

La similitud se basa en dos cosas diferentes: nombres propios (para los documentos más cercanos) y un vocabulario político general (para los más lejanos).

### Documento 5

In [63]:
doc_4_similar, doc_4_scores = get_similar_documents(static_document_list[4])



--- DOCUMENTO ORIGINAL ---
DOC: 4669
CLASS: sci.electronics


:      Hello, 
: 
:    Let me introduce a problem:
: 
:    When I measure a sinusoidal wave (voltage) with a digital voltmeter, using 
: AC mode, my output is an rms value (a peak value over 2 squared). / Right? / 
:    When I measure a square wave in the same mode (AC), my output is equal 
: to a peak value, actually, to the upper flat boundary of the wave.
:    I assumed, that a digital voltmeter makes some kind of integration of the 
: input value, and divides it over the wave period. / Right?/
:    Now, I used it to measure the same  square wave as above, but distorted 
: by high-frequency harmonics. Ideally, output should be the same, but...
: The output value was only about 10% of the previous one! 
:    Why? What is the nature of this output value? What does the voltmeter 
: actually measure? And what does it show?  


Re: RMS readings

Unless the DVM *says* it's measuring RMS, it's probably Average voltage.
The inp

El primer documento que recupera (4270) es muy similar. El segundo documento y el cuarto hay una pequeña confusión intuyo que es por la onda cuadrada "Square Wave" vs "wave" de saludar. El tercer y quinto documento hablan de valores espeficios de electronica.